In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

#import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Import packages.

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
!pip install -qU python-gdcm pydicom pylibjpeg
!pip install python-gdcm
!pip install -U python-gdcm
!pip install scikit-image


In [ ]:
from functools import partial
import multiprocessing as mp
from pathlib import Path
import numpy as np
import pandas as pd
import pydicom
from skimage.transform import resize
import tqdm
import matplotlib.pyplot as plt 

import gdcm
import pylibjpeg


Done by Ramesh for variable values

In [ ]:
# RESCALE_TO = (512, 512)
RESCALE_TO = (256, 256)
DATA_PATH = Path("/kaggle/input/rsna-breast-cancer-detection")
TRAINING_IMAGES = DATA_PATH.joinpath("train_images").iterdir()
TRAINING_DATA = DATA_PATH.joinpath("train.csv")
TEST_IMAGES = DATA_PATH.joinpath("test_images").iterdir()
TEST_DATA = DATA_PATH.joinpath("test.csv")

reading the train.csv file.

In [ ]:
training_data_df = pd.read_csv(TRAINING_DATA)
training_data_df

Loading 100 images in cancerous and 100 in non cancerous.

In [ ]:
cancerous =training_data_df[(training_data_df['cancer']==1)].tail(100)
non_cancerous=training_data_df[(training_data_df['cancer']==0)].head(100)
tr_img=pd.concat([cancerous, non_cancerous],axis=0)
tr_img

Done by Ramesh. Danya called them directly in the block.

In [ ]:
def dicom_file_to_data(path):
    dicom = pydicom.read_file(path)
    data = dicom.pixel_array
    resized_image = resize(data, (RESCALE_TO))
    data = (resized_image -resized_image.min()) / (resized_image.max() - resized_image.min())
    if dicom.PhotometricInterpretation == "MONOCHROME1":
        data = 1 - data
    return data * dicom.RescaleSlope + dicom.RescaleIntercept  #Already in hounse field unit

In [ ]:
# This block load the images for the patients in tr_img(both cancerous and non-cancerous data).
# Created Empty list
TR_IMAGES = []
TR_DATA = []

# Loop for all the rows of Data array - 
#tr_img.iterrows- rows in the data array
for i,row in tqdm.tqdm(tr_img.iterrows()):
    patient_id = row['patient_id']
    image_id = row['image_id']
    original_path='/kaggle/input/rsna-breast-cancer-detection/train_images'
    img_path =  f'{original_path}/{patient_id}/{image_id}.dcm'
    
    
    dicom = pydicom.read_file(img_path)
    data = dicom.pixel_array
    # Resize to 256X256
    resized_image = resize(data, (RESCALE_TO))
    # bring the number range to 0-1 for the image data
    # TBD- Pre-Process the image to filter the body part and other operations.
    
    data = (resized_image -resized_image.min()) / (resized_image.max() - resized_image.min())
    if dicom.PhotometricInterpretation == "MONOCHROME1":
        data = 1 - data
    data=data * dicom.RescaleSlope + dicom.RescaleIntercept  #Already in hounse field unit
   
    TR_IMAGES.append(data)
    TR_DATA.append(tr_img['cancer'].iloc[0])

In [ ]:
type(TR_IMAGES)
#print(len(TR_DATA))


In [ ]:
# Convert the list to array
TR_IMAGES=np.array(TR_IMAGES)
TR_DATA= np.array(TR_DATA)

In [ ]:
type(TR_IMAGES)

In [ ]:
!pip install scikit-learn

In [ ]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.preprocessing import image
from sklearn.model_selection import train_test_split
import numpy as np


In [ ]:
X_train = X_train.astype("float32")
X_test = X_test.astype("float32")

y_train = y_train.astype("float32")
y_test = y_test.astype("float32")

In [ ]:
model = Sequential()
model.add(Conv2D(8, kernel_size=(3, 3),activation='relu',input_shape=(256,256,1)))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64,(3, 3), activation='relu'))
model.add(Dropout(0.25))

model.add(Flatten())
# original coded by danya for dense = 100
model.add(Dense(10, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='softmax'))

X_train, X_test, y_train, y_test = train_test_split(TR_IMAGES, TR_DATA,shuffle=True)
# Metrics - For classification use accuracy and for regression use RMSE value.
model.compile(loss='categorical_crossentropy',optimizer='Adam',metrics=['accuracy'])


#X_train = np.asarray(X_train).astype(np.float32)
#X_test   = np.asarray(X_test).astype(np.float32)
X_train=tf.convert_to_tensor(X_train, dtype=tf.float32)

X_train = np.expand_dims(X_train, axis=-1)
X_test=np.expand_dims(X_test, axis=-1)

In [ ]:
# to pass validation
X_val = X_train[-20:]
y_val = y_train[-20:]

In [ ]:
X_train = X_train[:-20]
y_train = y_train[:-20]

In [ ]:
print(len(X_train))
print(len(y_train))
print(len(X_test))
print(len(y_test))
print(len(X_val))
print(len(y_val))
type(y_train)

In [ ]:
model.summary()

In [ ]:
with tf.device('/GPU:0'):
    model.fit(X_train,y_train, epochs=20, verbose=2, validation_data=(X_val,y_val))
              
#model.fit(X_train,y_train, epochs=20, verbose=2)# validation_data=(X_test, y_test))

In [ ]:
results = model.evaluate(X_test, y_test)
results

In [ ]:
y_pred = model.predict(X_test)
y_pred

In [ ]:
len(y_pred)


In [ ]:
len(y_test)

In [ ]:
import sklearn.metrics as metrics

In [ ]:
confusion_matrix = metrics.confusion_matrix(y_test, y_pred) 

In [ ]:
cm_display = metrics.ConfusionMatrixDisplay(confusion_matrix = confusion_matrix, display_labels = [False, True])

cm_display.plot()
plt.show()